In [3]:
import pandas as pd
import numpy as np

In [7]:
# read in the data
df = pd.read_csv('/home/dustin/Documents/Study/Master2/CILProject22/data_raw/data_train.csv')
df[:100]

,Id,Prediction
0,r44_c1,4
1,r61_c1,3
2,r67_c1,4
3,r72_c1,3
4,r86_c1,5
...,...,...
95,r2706_c1,4
96,r2820_c1,3
97,r2883_c1,2
98,r2939_c1,3


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch as nn
# import sigmoid_layer as sl
import torch.nn.functional as F


class model(pl.LightningModule):
    def __init__(self):
        super(model,self).__init__()
        self.phi = nn.Linear(11000,10)
        self.phi_IC = nn.Linear(11000,10)
        self.phi_UC = nn.Linear(11000,10)
        self.lr = 0.01
        self.optimizer = nn.optim.Adam(self.parameters(), lr=self.lr)
        
    def forward(self, x):
        self.phi(x)
        self.phi_IC(x)
        self.phi_UC(x)
        return self.phi(x), self.phi_IC(x), self.phi_UC(x)
    
    def configure_optimizers(self):
        return nn.optim.Adam(self.parameters(), lr=self.lr)
       
    def training_step(self, train_batch, batch_idx):
        x, ratings = train_batch
        phi, phi_IS, phi_UC = self.forward(x)
        loss = self.loss(phi, phi_IS, phi_UC, ratings)
        return loss
        
    def validation_step(self, valid_batch, batch_idx):
        x, y = valid_batch
        logits = self.forward(x)
        loss = self.loss(logits,y)

    def loss(self, phis, phis_IS, phis_UC, ratings): # implement the loss function
        alpha = 0.1
        lam = 0.1
        reg = 1.0
        loss_DS = [F.logsigmoid(phi[0] * phi[1]) for phi in phis].mean()
        loss_NS = [F.logsigmoid(phi[0] * phis_IS[2]) for phi in phis].mean()
        # norm =    only for overfitting
        loss = alpha*(loss_DS + lam*loss_NS) # + reg*norm)
        return loss

In [ ]:
# Create Model Object
clf = model()
# Create Data Module Object
train_loader = DataLoader(df, batch_size=32)
val_loader = DataLoader(df, batch_size=32)
# Create Trainer Object
trainer = pl.Trainer(gpus=1,accelerator='dp',max_epochs=5)
trainer.fit(clf,train_dataloader=train_loader,val_dataloader=val_loader)